In [1]:
import sys
sys.path.append('./airflow')

import os
from functions.images.detr.util.features import get_detr_model, extract_detr_features
from functions.images.yolo.util.features import extract_yolo_features, get_Net_yolov4
from functions.images.detr.util.transform import reshape

import pymongo
import json
import requests
import numpy as np
from tqdm import tqdm

In [2]:
with open("./airflow/config/env.json", "r") as file:
    config = json.load(file)
    mongo_url = config['mongodb']['MONGO_ATLAS_PYTHON_GCP']

In [ ]:
# Tải mô hình DETR
detr_model, postprocessor = get_detr_model(pretrained=True)
yolo_model = get_Net_yolov4("./airflow/functions/images/yolo/model/yolov4.weights", "./airflow/functions/images/yolo//model/yolov4.cfg")
# Khởi tạo một dictionary để lưu trữ các đặc trưng của ảnh
image_features = {}

# Lặp qua từng ảnh trong thư mục
with pymongo.MongoClient(mongo_url) as client:
    db = client['imcp']
    pipeline = [{
            '$sort': {'url': 1}
        }, {
            '$project': {'created_time': 0, 'publisher': 0, '_id': 0 }
        }, {
            '$limit': 8000
        }
    ]
    documents = db['refined'].aggregate(pipeline)
    image_cache = {}
    for doc in tqdm(documents):
        url = doc['url']
        if url not in image_cache:
            response = requests.get(url)
            image_cache[url] = response.content

        # # Trích xuất đặc trưng từ YOLO
        yolo_features = extract_yolo_features(yolo_model, image_cache[url])
        doc['yolov4_encode'] = yolo_features
        
        # Trích xuất đặc trưng từ DETR
        image_tensor = reshape(image_cache[url])
        detr_features = extract_detr_features(image_tensor, detr_model)
        doc['detr_encode'] = detr_features
        
        # features = combine_features(yolo_features, detr_features)
        print(doc['yolov4_encode'].shape)
        # break

In [ ]:
# %pip install scipy